In [16]:
import pandas as pd
from gensim import corpora, models
import pandas as pd
import numpy as np
import sys
from collections import Counter
sys.path.append("c:/users/david/desktop/sandbox/lsi-tagger")
from model.tag_extractor import TagExtractor
from model.text_cleaner import TextCleaner

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
df = pd.read_csv('data/h&m_kaggle_products.csv')
df = df[~df.isnull().any(axis=1)]
print(df.shape)
df.head(2)

(105126, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [5]:
documents = df['detail_desc'].drop_duplicates().values
cleaned_documents = TextCleaner(word_count_min=2, word_length_min=2).clean_documents(documents)
len(cleaned_documents)

100%|██████████████████████████████████████████████████████████████████████████| 43404/43404 [00:04<00:00, 9768.30it/s]


43404

In [9]:
# Remove empty documents
doc_keep_inds = np.array([n for n,doc in enumerate(cleaned_documents) if len(doc)>0])
cleaned_documents = np.array(cleaned_documents)[doc_keep_inds].tolist()
documents = np.array(documents)[doc_keep_inds].tolist()

len(documents), len(cleaned_documents)

C:\Users\David\AppData\Local\Temp\ipykernel_11928\2820098043.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cleaned_documents = np.array(cleaned_documents)[doc_keep_inds].tolist()


(43395, 43395)

In [10]:
# LSI (https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html)
  # https://radimrehurek.com/gensim/models/lsimodel.html
dictionary = corpora.Dictionary(cleaned_documents)
corpus = [dictionary.doc2bow(doc) for doc in cleaned_documents]

len(dictionary)

2003

In [11]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=200)
corpus_lsi = lsi_model[corpus_tfidf]

In [12]:
lsi_model.print_topics(5)

[(0,
  '0.179*"long" + 0.178*"sleeves" + 0.168*"soft" + 0.166*"short" + 0.165*"cuffs" + 0.163*"hem" + 0.161*"pockets" + 0.159*"cotton" + 0.153*"jersey" + 0.148*"waist"'),
 (1,
  '-0.234*"knit" + 0.232*"pockets" + -0.223*"ribbing" + -0.211*"jumper" + 0.206*"fly" + -0.198*"neckline" + -0.183*"long" + 0.174*"legs" + -0.174*"hem" + -0.164*"cuffs"'),
 (2,
  '0.233*"dress" + 0.205*"seam" + -0.202*"ribbing" + 0.195*"neck" + -0.191*"pockets" + 0.179*"short" + -0.177*"sweatshirt" + 0.165*"skirt" + 0.151*"unlined" + 0.149*"weave"'),
 (3,
  '0.332*"imitation" + 0.323*"leather" + 0.286*"insoles" + 0.286*"soles" + 0.256*"linings" + 0.245*"cm" + 0.220*"rubber" + 0.193*"loop" + 0.154*"heel" + 0.139*"lacing"'),
 (4,
  '0.280*"jersey" + -0.263*"collar" + -0.221*"buttons" + 0.206*"elasticated" + -0.190*"buttoned" + -0.180*"jacket" + -0.179*"chest" + 0.164*"soft" + -0.159*"flap" + -0.156*"cuffs"')]

In [13]:
def lsi_corpus2vec(lsi_corpus_vec):
    vec = np.zeros(200)
    for ind, value in lsi_corpus_vec:
        vec[ind] = value
    return vec

lsi_vecs = np.array([lsi_corpus2vec(cl) for cl in corpus_lsi])
lsi_vecs_normed = lsi_vecs/np.linalg.norm(lsi_vecs, axis=1)[:,None]

In [14]:
lsi_topic_matrix = lsi_model.get_topics()

def extract_tags(input_ind, candidate_ind, n_tags=5):

    # Get shared words using the sparse TF-IDF corpus
    shared_word_ids = np.array(list(set(np.array(corpus_tfidf[input_ind])[:,0]) \
                                  & set(np.array(corpus_tfidf[candidate_ind])[:,0]))).astype(int)

    # Get word-topic vectors for those shared words
    shared_word_topics = lsi_topic_matrix.T[shared_word_ids]

    # Sum those word-topic vectors weighed by each document's topic loadings
    shared_lsi_word_loadings = np.sum((shared_word_topics*np.array(corpus_lsi[input_ind])[:,1]) \
                                    + (shared_word_topics*np.array(corpus_lsi[candidate_ind])[:,1]), axis=1)

    # Rank shared words by highest weighed word-topic loadings
    ranking = np.argsort(-shared_lsi_word_loadings)
    ranked_shared_word_ids = shared_word_ids[ranking]

    tags = [dictionary[r] for r in ranked_shared_word_ids][:n_tags]
    scores = shared_lsi_word_loadings[ranking][:n_tags]
    return tags, scores

def show_recommendations_with_tags(input_ind, internal_n_recs=50, show_n_recs=5,
                                   n_input_tags=10, n_candidate_tags=5):
    sims = lsi_vecs_normed[input_ind].dot(lsi_vecs_normed.T)
    top_inds = np.argsort(-sims)[:internal_n_recs]

    results = []
    all_tags = []
    for candidate_ind in top_inds:
        tags, scores = extract_tags(input_ind, candidate_ind, n_tags=n_candidate_tags)
        results.append([candidate_ind, tags, scores])
        all_tags.extend(tags)

    print(documents[input_ind])
    print(Counter(all_tags).most_common(n_input_tags))
    print('='*150)
    print()

    for data in results[:show_n_recs]:
        print(documents[data[0]])
        display(list(zip(data[1], data[2])))
        print('-'*150)

In [17]:
# TODO: Ability to re-rank by selecting tags
show_recommendations_with_tags(10000, internal_n_recs=50, show_n_recs=5,
                               n_input_tags=10, n_candidate_tags=5)

Mesh hi-tops with rubber details, a rib-knit shaft with a wide elastic strap over the foot, and a loop at the back. Mesh insoles and rubber soles.
[('mesh', 50), ('soles', 49), ('loop', 46), ('insoles', 33), ('rubber', 32), ('tops', 31), ('elastic', 4), ('rib', 2), ('hi', 2), ('strap', 1)]

Mesh hi-tops with rubber details, a rib-knit shaft with a wide elastic strap over the foot, and a loop at the back. Mesh insoles and rubber soles.


[('mesh', 0.7141216636722489),
 ('tops', 0.5836421685610182),
 ('rib', 0.4333607306618575),
 ('soles', 0.3941848233532636),
 ('rubber', 0.374554817370311)]

------------------------------------------------------------------------------------------------------------------------------------------------------
Mesh hi-tops with rubber details, and a rib-knit shaft with a loop at the back and lacing at the front. Mesh linings and insoles and chunky rubber soles.


[('mesh', 0.733082668674957),
 ('tops', 0.5844671799428358),
 ('rib', 0.434034138792625),
 ('soles', 0.41102535058882755),
 ('loop', 0.39970960714371945)]

------------------------------------------------------------------------------------------------------------------------------------------------------
Scuba fabric hi-tops with woven sections, reflective details, a zip on each side and a loop at the back. Mesh linings and insoles and rubber soles.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
Hi-tops in mesh and scuba fabric with lacing at the front and loops front and back. EVA (ethylene-vinyl acetate) and rubber soles.


[('mesh', 0.5541892809320705),
 ('tops', 0.5350643417458049),
 ('soles', 0.3085499682650582),
 ('rubber', 0.29176841990369773),
 ('hi', 0.150032595401553)]

------------------------------------------------------------------------------------------------------------------------------------------------------
Hi-tops in imitation leather with a scuba shaft, loop front and back, mesh insoles and rubber soles.


[('tops', 0.6446047523745317),
 ('mesh', 0.5784170302062039),
 ('loop', 0.41264903088190463),
 ('soles', 0.3687503241957863),
 ('rubber', 0.33900985231446545)]

------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
class TagExtractor:
    def __init__(self, 
                 word_count_min=2, 
                 word_length_min=2, 
                 num_lsi_topics=300):
        self.word_count_min = word_count_min
        self.word_length_min = word_length_min
        self.num_lsi_topics = num_lsi_topics
    
    def fit(self, documents):
        # Clean text
        # TODO: TextCleaner needs a fit and transform method so that I can look up word counts for new documents
        self.tc = TextCleaner(word_count_min=self.word_count_min, 
                              word_length_min=self.word_length_min)
        cleaned_documents = self.tc.clean_documents(documents)
            
        # Create document lookup
        self.problem_docs = []
        self.doc2ind = {}
        for n, doc, cleaned_doc in enumerate(zip(documents, cleaned_documents)):
            if len(cleaned_doc)==0:
                self.problem_docs.append(doc)
            doc2ind[doc] = n
                
        # Warn for empty documents
        if len(self.problem_docs) > 0:
            print("""Warning: Some documents yield no clean tokens. 
            These documents won't have tags.
            Check self.problem_docs for more detail.""")
        
        # Train TF-IDF
        self.dictionary = corpora.Dictionary(cleaned_documents)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in cleaned_documents]
        self.tfidf = models.TfidfModel(self.corpus)
        self.corpus_tfidf = self.tfidf[self.corpus]

        # Train LSI
        self.lsi_model = models.LsiModel(self.corpus_tfidf, 
                                         id2word=self.dictionary, 
                                         num_topics=self.num_lsi_topics)
        self.corpus_lsi = self.lsi_model[self.corpus_tfidf]
        
        # Save the topic matrix for tag extraction
        self.lsi_topic_matrix = self.lsi_model.get_topics()
        
    def _transform(self, document):
        doc_ind = doc2ind.get(document) 
        if doc_ind is None:
            # TODO: TextCleaner needs a fit and transform method so that I can look up word counts for new documents
            cleaned_documents = self.tc.clean_documents([document])
            corpus = self.dictionary.doc2bow(cleaned_documents[0])
            corpus_tfidf = self.tfidf[corpus]
            corpus_lsi = self.lsi_model[corpus_tfidf] # TODO: Can I just add this to self.corpus_lsi so it gets added online? Same for TF-IDF
        else:
            corpus_tfidf = self.corpus_tfidf[doc_ind]
            corpus_lsi = self.corpus_lsi[doc_ind]
        return corpus_tfidf, corpus_lsi
        
    def extract_tags(self, document_a, document_b, n_tags):
        corpus_tfidf_a, corpus_lsi_a = self.transform(document_a)
        corpus_tfidf_b, corpus_lsi_b = self.transform(document_b)
        
        # Get shared words using the sparse TF-IDF corpus
        shared_word_ids = np.array(list(set(np.array(corpus_tfidf_a)[:,0]) \
                                      & set(np.array(corpus_tfidf_b)[:,0]))).astype(int)

        # Get word-topic vectors for those shared words
        shared_word_topics = self.lsi_topic_matrix.T[shared_word_ids]

        # Sum those word-topic vectors weighed by each document's topic loadings
        shared_lsi_word_loadings = np.sum((shared_word_topics*np.array(corpus_lsi_a)[:,1]) \
                                        + (shared_word_topics*np.array(corpus_lsi_b)[:,1]), axis=1)

        # Rank shared words by highest weighed word-topic loadings
        ranking = np.argsort(-shared_lsi_word_loadings)
        ranked_shared_word_ids = shared_word_ids[ranking]

        tags = [self.dictionary[r] for r in ranked_shared_word_ids][:n_tags]
        scores = shared_lsi_word_loadings[ranking][:n_tags]
        return tags, scores